#Collaborative Filtering

In [7]:
import pandas as pd

df_shared = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/shared_articles.csv')
df_user = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/users_interactions.csv')

df_user.head()


,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


In [8]:
df_articles = df_shared.drop(columns='timestamp')
df_interactions = df_user.drop(columns='timestamp')

df_interactions = df_interactions[df_interactions['eventType'].isin(['VIEW', 'LIKE', 'BOOKMARK', 'FOLLOW', 'COMMENT'])]
df_interactions = df_interactions.drop_duplicates(subset=['personId', 'contentId'], keep='last')

user_activity = df_interactions.groupby('personId').size()
active_users = user_activity[user_activity >= 3].index
df_interactions = df_interactions[df_interactions['personId'].isin(active_users)]

value_counts = df_interactions['contentId'].value_counts()
keep_list = value_counts[value_counts >= 9]
df_interactions_reduced = df_interactions[df_interactions['contentId'].isin(keep_list.index)]

In [9]:
print(f"Number of interactions: {len(df_interactions)}")
print(f"Number of articles: {df_interactions.contentId.nunique()}")
print(f"Number of users: {df_interactions.personId.nunique()}")

sparsity = 1 - (len(df_interactions)) / (df_interactions.contentId.nunique() * df_interactions.personId.nunique())
print(f"Sparsity: {sparsity:.4f}")

Number of interactions: 39995
Number of articles: 2985
Number of users: 1400
Sparsity: 0.9904


In [10]:
print('Min:\t\t', df_interactions['contentId'].value_counts().min())
print('Quartile 1:\t', df_interactions['contentId'].value_counts().quantile(0.25))
print('Median:\t\t', df_interactions['contentId'].value_counts().quantile(0.5))
print('Quartile 3:\t', df_interactions['contentId'].value_counts().quantile(0.75))
print('Max:\t\t', df_interactions['contentId'].value_counts().max())

Min:		 1
Quartile 1:	 4.0
Median:		 9.0
Quartile 3:	 17.0
Max:		 237


In [11]:
print('Min:\t\t', df_interactions['personId'].value_counts().min())
print('Quartile 1:\t', df_interactions['personId'].value_counts().quantile(0.25))
print('Median:\t\t', df_interactions['personId'].value_counts().quantile(0.5))
print('Quartile 3:\t', df_interactions['personId'].value_counts().quantile(0.75))
print('Max:\t\t', df_interactions['personId'].value_counts().max())


Min:		 3
Quartile 1:	 5.0
Median:		 11.0
Quartile 3:	 27.0
Max:		 961


In [12]:
from scipy.sparse import csr_matrix
import numpy as np

U = df_interactions_reduced['personId'].nunique()
I = df_interactions_reduced['contentId'].nunique()

user_mapper = dict(zip(np.unique(df_interactions_reduced['personId']), list(range(U))))
item_mapper = dict(zip(np.unique(df_interactions_reduced['contentId']), list(range(I))))

user_inv_mapper = dict(zip(list(range(U)), np.unique(df_interactions_reduced['personId'])))
item_inv_mapper = dict(zip(list(range(I)), np.unique(df_interactions_reduced['contentId'])))

user_index = [user_mapper[i] for i in df_interactions_reduced['personId']]
item_index = [item_mapper[i] for i in df_interactions_reduced['contentId']]

X = csr_matrix((np.ones(len(user_index)), (item_index, user_index)), shape=(I, U))

In [13]:
from sklearn.neighbors import NearestNeighbors

def recommend(itemId, X, item_mapper, item_inv_mapper, k, metric='cosine', messages=True):
    rec_ids = []

    item = item_mapper[itemId]
    item_vector = X[item]

    knn = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric).fit(X)
    rec = knn.kneighbors(item_vector.reshape(1, -1), return_distance=True)

    rec_indices = rec[1][0][1:]
    rec_distances = rec[0][0][1:]

    for idx in rec_indices:
        rec_ids.append(item_inv_mapper[idx])

    if messages:
        print(f'List of recommended item indices:\n{rec_indices}\n')
        print(f'List of recommended contentIds:\n{rec_ids}\n')
        print(f'Similarity distances:\n{rec_distances}\n')

    return rec_ids, rec_distances

In [14]:
df_articles.set_index('contentId', inplace=True)

article_id = df_interactions_reduced['contentId'].iloc[0]

rec_ids, rec_distances = recommend(article_id, X, item_mapper, item_inv_mapper, k=5)

original_title = df_articles.loc[article_id, 'title']
print(f"If you like article ID {article_id} ({original_title}), you may also enjoy:\n")

valid_rec_ids = [rid for rid in rec_ids if rid in df_articles.index]

print(df_articles.loc[valid_rec_ids][['title']])

List of recommended item indices:
[1234  612  648  115  107]

List of recommended contentIds:
[np.int64(5688279681867464747), np.int64(-1625217607550912638), np.int64(-1196068832249300490), np.int64(-8018088591294066196), np.int64(-8142426490949346803)]

Similarity distances:
[0.49748109 0.5545646  0.59798487 0.61509982 0.65573481]

If you like article ID 3460026829794173084 (TAM recebe primeira aeronave com o design e pintura da LATAM e avança na implantação da nova marca!), you may also enjoy:

                                                                  title
contentId                                                              
 5688279681867464747  Margaret Gould Stewart: How giant websites des...
-1625217607550912638  Google made a huge change to the way Google Ma...
-1196068832249300490                  The 'impossible' is now a reality
-8018088591294066196      LOGOBR School | Behance Reviews Campinas 2016
-8142426490949346803  A identidade visual de Budapeste para as Oli

In [ ]:
recommendations_df = pd.DataFrame({
    'input_contentId': [article_id] * len(rec_ids),
    'recommended_contentId': rec_ids,
    'similarity': rec_distances
})

recommendations_df.to_csv('collaborative_recommendations.csv', index=False)

#Content Filtering

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


df_shared = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/shared_articles.csv')
df_user = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/users_interactions.csv')
print(df_shared.columns)
df_shared.head()

Index(['timestamp', 'eventType', 'contentId', 'authorPersonId',
       'authorSessionId', 'authorUserAgent', 'authorRegion', 'authorCountry',
       'contentType', 'url', 'title', 'text', 'lang'],
      dtype='object')


,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [3]:
print(df_shared.shape)
df_shared.isna().sum()

(3122, 13)


,0
timestamp,0
eventType,0
contentId,0
authorPersonId,0
authorSessionId,0
authorUserAgent,2442
authorRegion,2442
authorCountry,2442
contentType,0
url,0


In [4]:
df_shared.authorUserAgent.fillna('unknown', inplace=True)
df_shared.authorRegion.fillna('unknown', inplace=True)
df_shared.authorCountry.fillna('unknown', inplace=True)

# Very important step
df_shared.reset_index(inplace=True)

print(df_shared.isna().sum(), '\n')
df_shared.shape

index              0
timestamp          0
eventType          0
contentId          0
authorPersonId     0
authorSessionId    0
authorUserAgent    0
authorRegion       0
authorCountry      0
contentType        0
url                0
title              0
text               0
lang               0
dtype: int64 



<ipython-input-4-4a39909e5268>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_shared.authorUserAgent.fillna('unknown', inplace=True)
<ipython-input-4-4a39909e5268>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', t

(3122, 14)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TfidfVectorizer and Remove stopwords
tfidf = TfidfVectorizer(stop_words='english')

# Fit and transform the data to a tfidf matrix
tfidf_matrix = tfidf.fit_transform(df_shared['text'])

# Print the shape of the tfidf_matrix
print(tfidf_matrix.shape)

# Preview the matrix by placing it into a DataFrame (which we won't need later)
#change this to text
df_tfidf = pd.DataFrame(tfidf_matrix.T.todense(), index=tfidf.get_feature_names_out(), columns=df_shared['text'])
df_tfidf

(3122, 72353)


text          All of this work is still very early. The first full public version of the Ethereum software was recently released, and the system could face some of the same technical and legal problems that have tarnished Bitcoin. Many Bitcoin advocates say Ethereum will face more security problems than Bitcoin because of the greater complexity of the software. Thus far, Ethereum has faced much less testing, and many fewer attacks, than Bitcoin. The novel design of Ethereum may also invite intense scrutiny by authorities given that potentially fraudulent contracts, like the Ponzi schemes, can be written directly into the Ethereum system. But the sophisticated capabilities of the system have made it fascinating to some executives in corporate America. IBM said last year that it was experimenting with Ethereum as a way to control real world objects in the so-called Internet of things. Microsoft has been working on several projects that make it easier to use Ethereum on its computing cloud, Azure. "Ethereum is a general platform where you can solve problems in many industries using a fairly elegant solution - the most elegant solution we have seen to date," said Marley Gray, a director of business development and strategy at Microsoft. Mr. Gray is responsible for Microsoft's work with blockchains, the database concept that Bitcoin introduced. Blockchains are designed to store transactions and data without requiring any central authority or repository. Blockchain ledgers are generally maintained and updated by networks of computers working together - somewhat similar to the way that Wikipedia is updated and maintained by all its users. Many corporations, though, have created their own Ethereum networks with private blockchains, independent of the public system, and that could ultimately detract from the value of the individual unit in the Ethereum system - known as an Ether - that people have recently been buying. The interest in Ethereum is one sign of the corporate fascination with blockchains. Most major banks have expressed an interest in using them to make trading and money transfer faster and more efficient. On Tuesday, executives from the largest banks will gather for a conference, "Blockchain: Tapping I nto the Real Potential , Cutting Through the Hype." Many of these banks have recently been looking at how some version of Ethereum might be put to use. JPMorgan, for instance, has created a specific tool, Masala, that allows some of its internal databases to interact with an Ethereum blockchain. Michael Novogratz, a former top executive at the private equity firm Fortress Investing Group, who helped lead Fortress's investment in Bitcoin, has been looking at Ethereum since he left Fortress last fall. Mr. Novogratz said that he made a "significant" purchase of Ether in January. He has also heard how the financial industry's chatter about the virtual currency has evolved. "A lot of the more established players were thinking, 'It's still an experiment,' " he said. "It feels like in the last two to three months that experiment is at least getting a lot more validation." Since the beginning of the year, the value of an individual unit of Ether has soared as high as $12 from around $1. That has brought the value of all existing Ether to over $1 billion at times, significantly more than any virtual currency other than Bitcoin, which had over $6 billion in value outstanding last week. Since Bitcoin was invented, there have been many so-called alt-coins that have tried to improve on Bitcoin, but none have won the following of Ethereum. Unlike Bitcoin, which was released in 2009 by a mysterious creator known as Satoshi Nakamoto, Ethereum was created in a more transparent fashion by a 21-year-old Russian-Canadian, Vitalik Buterin, after he dropped out of Waterloo University in Ontario. The most basic aim of Ethereum was to make it possible to program binding agreements into the blockchain - the smart contract concept. Two people, for i

In [6]:
from sklearn.metrics.pairwise import linear_kernel

#this table is what you need to make predictions
# Compute the cosine similarity between each text
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# For easier viewing, put it in a dataframe
df_results = pd.DataFrame(cosine_sim, columns=df_shared['contentId'], index=df_shared['contentId'])
df_results.to_csv('content_filtering_results.csv')
df_results

contentId,-6451309518266745024,-4110354420726924665,-7292285110016212249,-6151852268067518688,2448026894306402386,-2826566343807132236,-2148899391355011268,4119190424078847945,-7926018713416777892,3353902017498793780,...,-5953227649059336919,-3201623914150480647,-4132331404553626868,1908114740148100507,4675505028897335428,9213260650272029784,-3295913657316686039,3618271604906293310,6607431762270322325,4109618890343020064
contentId,,,,,,,,,,,,,,,,,,,,,
-6451309518266745024,1.000000,1.000000,0.029999,0.016192,0.234378,0.102207,0.186021,0.150986,0.182221,0.865994,...,0.043244,0.024061,0.007743,0.023067,0.011399,0.003296,0.021317,0.051052,0.090121,0.020183
-4110354420726924665,1.000000,1.000000,0.029999,0.016192,0.234378,0.102207,0.186021,0.150986,0.182221,0.865994,...,0.043244,0.024061,0.007743,0.023067,0.011399,0.003296,0.021317,0.051052,0.090121,0.020183
-7292285110016212249,0.029999,0.029999,1.000000,0.023368,0.032324,0.023549,0.041352,0.033399,0.042882,0.031538,...,0.019542,0.006460,0.006152,0.015815,0.003797,0.000000,0.010076,0.026958,0.033152,0.011318
-6151852268067518688,0.016192,0.016192,0.023368,1.000000,0.018691,0.031610,0.007079,0.013496,0.023717,0.015807,...,0.050553,0.018448,0.007774,0.035578,0.024903,0.000000,0.077019,0.022694,0.045090,0.000000
2448026894306402386,0.234378,0.234378,0.032324,0.018691,1.000000,0.214121,0.159303,0.231415,0.143972,0.186033,...,0.038043,0.022978,0.009663,0.050044,0.024246,0.013207,0.020210,0.026150,0.094685,0.011511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9213260650272029784,0.003296,0.003296,0.000000,0.000000,0.013207,0.000000,0.012865,0.000000,0.002807,0.002543,...,0.000000,0.006506,0.000000,0.000000,0.000000,1.000000,0.000000,0.000125,0.012346,0.000000
-3295913657316686039,0.021317,0.021317,0.010076,0.077019,0.020210,0.034869,0.011329,0.018790,0.032870,0.015953,...,0.074052,0.049896,0.008562,0.034565,0.023315,0.000000,1.000000,0.029901,0.039184,0.008100
3618271604906293310,0.051052,0.051052,0.026958,0.022694,0.026150,0.044007,0.017267,0.025773,0.019949,0.042678,...,0.041965,0.019718,0.026403,0.024435,0.027546,0.000125,0.029901,1.000000,0.059641,0.079239
